## Cardiac MRF Workshop 2022-A

This notebook covers what is being done in `prep_sim` in `cMRF_Workshop2022_helper`. Here we will load in the segmentation and motion vector fields, check what they look like and if they match. We will also add some information about the orientation to make sure this is correctly defined. 

For this notebook we need:

- an anatomy segmentation in RAI format (right-anterior-inferior)
- motion information (cardiac or respiratory) matching the anatomy segmentation

In [ ]:
# Import required packages
import shutil, os
from pathlib import Path 

import matplotlib.pyplot as plt

import nibabel as nib
import numpy as np

import sirf.Reg as pReg
import sirf.DynamicSimulation as pDS
import sirf.Gadgetron as pMR
from cil.utilities.jupyter import islicer, link_islicer

import auxiliary_functions as aux
import cMRF_Workshop2022_helper as helper

In [ ]:
# Set main paths
# Where is all the data
fpath_base = Path('/mnt/share/SA/')

# Where are we saving (intermediate) results
root_path = Path('/home/sirfuser/devel/Data_mrf_sim/')
root_path.mkdir(exist_ok=True, parents=True)


# These folders will be filled with the formatted input 
# for the simulation and will then also contain the output of the simulation
fpath_in = root_path / "Input"
fpath_in.mkdir(exist_ok=True, parents=True)
fpath_out = root_path / "Output"
fpath_out.mkdir(exist_ok=True, parents=True)

In [ ]:
# 3D tissue segmentation : nifti
fpath_segmentation_nii = fpath_base / 'XCat/label_volume_sa.nii'
# Respiratory motion fields : nifti
fpath_resp_mvf = fpath_base / 'XCat/mvf_resp/'
# Cardiac motion fields : nifti
fpath_card_mvf = fpath_base / 'XCat/mvf_card/'

## A Clean up folders

When we set up the simulation, we need to save several files temporarily. To make sure we don't run into any problems from previous simulations, we go through all the folders and clean them. When you run the following cell you will be asked if you want to delete any old content. Just say 'y' to agree.

In [ ]:
# Set up folders for simulation (and remove exisiting ones)
helper.clear_folders([fpath_in, fpath_out])

## B Segmentation

Now let's load the segmentation and check its size and visualise it. For the visualisation we are going to use `islicer`, which is an interactive viewer for 3D data provided by **CIL**. Of course you can also use `matplotlib` if you prefer. 

In [ ]:
fpath_segmentation_nii = fpath_base / 'XCat/label_volume_sa.nii'

segmentation_nii = nib.load(str(fpath_segmentation_nii))

# continue with segmentation as numpy array
segmentation = segmentation_nii.get_fdata()
print("The shape of the segmentation is {}".format(segmentation.shape))

Let's have a look at the data:

In [ ]:
islicer(segmentation, direction=2, origin='upper-left', cmap='viridis')

If you want to learn more about any of the SIRF or CIL functionality simply use the help command:

In [ ]:
help(islicer)

Ideally these data are already in RAI orientation.
RAI means: the data are available in memory such with increasing XYZ index the voxels move from left to right, from posteior to anterior, and from superior to inferior.

In [ ]:
# Before we continue we need to take a subset of slices, otherwise it will take too long
num_slices = 10
slice_start = 35
slice_end = slice_start + num_slices
slice_subset = range(slice_start,slice_end)

segmentation = segmentation[:,:,slice_subset]

print("Now our segmentation has the shape {}".format(segmentation.shape))

In [ ]:
islicer(segmentation, direction=2, origin='upper-left', cmap='viridis')

In [ ]:
# So far we only saw some voxelised data. Now we need to store it with approriate geometry information

# Define resolution
resolution_mm_per_pixel = np.array([2,2,-2,1])

# The first voxel center is lies at -FOV / 2 + dx/2
offset_mm =(-np.array(segmentation.shape)/2 + 0.5) * resolution_mm_per_pixel[0:3]

affine = np.diag(resolution_mm_per_pixel)
affine[:3,3] = offset_mm

print("We have an affine of \n {}".format(affine))

img = nib.Nifti1Image(segmentation, affine)
img.set_qform(affine) # crucial!
fname_segmentation = fpath_in / "segmentation.nii"
nib.save(img, str(fname_segmentation))


## C Motion vector fields

We have got the segmentation so now we need the motion vector fields. We will first load the respiratory motion fields and also select a subsect of slice as we did for the segmentation. Then we will calculate the amplitude of the motion between expiration and inspiration and compare it to the segmentation to make sure the motion vector fields actually fit to our segmentation. We will also add information about orienation and voxel size and save them for the simulation. 

Looking at the motion amplitude and comparing it to the segmentation is one way to check that the two fit, but we can also apply the motion vector fields to the segmentation to get an idea of our transformation. 

Finally we will repeat the above steps also for the cardiac motion fields. 

In [ ]:
# Path of the respiratory motion fields
fpath_resp_mvf = fpath_base / 'XCat/mvf_resp/'
resp_mvfs = aux.read_nii_motionfields(str(fpath_resp_mvf))

In [ ]:
# Let's find out what the size of the motion vector fields are
print(resp_mvfs.shape)

In [ ]:
# We also have to select a subset of the slice for the motion fields
resp_mvfs = resp_mvfs[:,:,:,slice_subset,:,:]
print("Now our motion fields have the shape {}".format(resp_mvfs.shape))

In [ ]:
# Now we plot the motion field amplitude and segmentation and see if they overlap
inhale_mvf = np.squeeze(resp_mvfs[-1,...])
inhale_abs = np.linalg.norm(inhale_mvf,axis=-1)
islicer(segmentation, direction=2, origin='upper-left', cmap='viridis')
islicer(inhale_abs, direction=2, origin='upper-left', cmap='viridis')

In [ ]:
# Store motion fields as nifti
fpath_mvf_output = fpath_in / 'mvfs_resp'
fpath_mvf_output.mkdir(exist_ok=True,parents=True)

aux.store_nii_mvfs(str(fpath_mvf_output), resp_mvfs, affine)
del resp_mvfs

In [ ]:
# Make a crosscheck to see the motion 
aux.deform_segmentation(str(fname_segmentation), str(fpath_mvf_output), str(fpath_out / "resp_motion_state_"))

In [ ]:
# Read in deformed segmentation
segm_resp = aux.read_deformed_nii_image(fpath_out, "resp_motion_state")

In [ ]:
# Visualise central slice
islicer(segm_resp[:,:,:,5], direction=0, origin='upper-left', cmap='viridis')

In [ ]:
# Same for cardiac motion fields
fpath_card_mvf = fpath_base / 'XCat/mvf_card/'
card_mvfs = aux.read_nii_motionfields(str(fpath_card_mvf))
card_mvfs = card_mvfs[:,:,:,slice_subset,:,:]

fpath_mvf_output = fpath_in / 'mvfs_card'
fpath_mvf_output.mkdir(exist_ok=True,parents=True)

aux.store_nii_mvfs(str(fpath_mvf_output), card_mvfs, affine)
del card_mvfs

In [ ]:
# Make a crosscheck to see the motion 
aux.deform_segmentation(str(fname_segmentation), str(fpath_mvf_output), str(fpath_out / "card_motion_state_"))
segm_card = aux.read_deformed_nii_image(fpath_out, "card_motion_state")
islicer(segm_card[:,:,:,5], direction=0, origin='upper-left', cmap='viridis')

In order to estimate what artefacts the cardiac motion is going to cause let's have a look at a simple average over all cardiac phases. We can also look at just an average over the first half (i.e. systolic phases). This  would roughly correspond to a higher heart rate (for an increase in the heart rate, systolic timings stay mostly constant and diastolic timings are shortened). We also focus on a region around the heart to better visualise what is going on. 

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,6))
ax[0].imshow(segm_card[0,100:200,100:200,5])
ax[0].set_title('Diastole (reference)')
ax[1].imshow(np.average(segm_card[:,100:200,100:200,5], axis=0))
ax[1].set_title('Average over all cardiac phases')
ax[2].imshow(np.average(segm_card[:5,100:200,100:200,5], axis=0))
ax[2].set_title('Average over sytolic cardiac phases');

## Recap

In this notebook we:

- loaded the segmentation and motion vector fields
- used `islicer` to explore the data
- added geometry information to a voxelised antatomy segmentation,
- ensured the motion information as displacement vector fields matched the anatomy,